In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, json

driver = webdriver.Chrome(executable_path="D:\chromedriver.exe")

url = 'https://www.snapdeal.com/products/mobiles-mobile-phones?sort=plrty'
driver.get(url)
time.sleep(3)

ulen, tlen = 0, 0
while True:
    for scroll in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_div = soup.find("div", {"id": "products"})
    page = main_div.find_all("div", {"class": "col-xs-24"})
    # print(len(page))

    for i in page:
        div = i.find("div", {"id": "pagination-txt"})
        ulen = int(div.find("span", {"id": "pagination-upper-count"}).text)
        tlen = int(div.find("span", {"id": "pagination-total-count"}).text)
    # print(ulen, tlen)
    if ulen == tlen: break
    else: pass

driver.quit()

section = main_div.find_all("section", {"data-dpwlbl": "Product Grid"})
links = []
for x, i in enumerate(section):
    in_div = i.find_all("div", {"class": ["col-xs-6  favDp product-tuple-listing js-tuple", "product-hover-state"]})
    for y, j in enumerate(in_div):
        a = j.find("a", {"target": "_blank"})
        links.append(a.get("href"))


print(len(links), links)    

159 ['https://www.snapdeal.com/product/redmi-9-prime-64gb-4/6917529666173669200#bcrumbLabelId:175', 'https://www.snapdeal.com/product/itel-a25-pro-32gb-2/8070451161033355439#bcrumbLabelId:175', 'https://www.snapdeal.com/product/redmi-note-9-128gb-4/6917529647638766635#bcrumbLabelId:175', 'https://www.snapdeal.com/product/blackberry-leap-16-gb-black/686616523201#bcrumbLabelId:175', 'https://www.snapdeal.com/product/redmi-note-9-64gb-4/7493990410611826392#bcrumbLabelId:175', 'https://www.snapdeal.com/product/yu-yu5012-16gb-3-gb/669097161560#bcrumbLabelId:175', 'https://www.snapdeal.com/product/lava-z61-pro-16gb-2/5188147437929984430#bcrumbLabelId:175', 'https://www.snapdeal.com/product/samsung-galaxy-m01s-32gb-3/6917529652083879877#bcrumbLabelId:175', 'https://www.snapdeal.com/product/oppo-mobile-cph1909-64gb-4/631723907397#bcrumbLabelId:175', 'https://www.snapdeal.com/product/redmi-8-64gb-4-gb/8070451181977327869#bcrumbLabelId:175', 'https://www.snapdeal.com/product/lava-z61-pro-16gb-2/

In [7]:
from selenium.webdriver.chrome.options import Options
import csv
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(executable_path="D:\chromedriver.exe", options=chrome_options)

row = ['Prod Name', 'Color', 'Rating', 'Price', 'RAM', 'Screen', 'Rear Camera', 'Front Camera', 'Memory', 'Battery', 'Processor']
file_object = open('snapdeal_csv.csv', 'a', encoding='utf-8', newline="")
writer = csv.DictWriter(file_object, fieldnames=row)
if file_object.tell() == 0:
    writer.writeheader()

details = {}

for url in links:
    print(url)
    driver.get(url)
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    indiv_prod = soup.find("div", {"class": "pdp-comp comp-product-description clearfix"})
    name = indiv_prod.find("h1", {"class": "pdp-e-i-head"}).get('title')
    s_name = ""
    for s in name:
        if s != ")": s_name += s
        else:
            s_name += ")"
            break
    details['prod_name'] = s_name
    details['color'] = str(indiv_prod.find("h1", {"class": "pdp-e-i-head"}).get('title')).split()[-1]
    try: details['rating'] = indiv_prod.find("span", {"itemprop": "ratingValue"}).text
    except: details['rating'] = "Currently Unavailable"
        
    try: details['price'] = str(indiv_prod.find("span", {"class": "pdp-final-price"}).text).lstrip("Rs.\xa0")
    except: details['price'] = "Currently Unavailable"
    
    indiv_prod_spec = soup.find("div", {"class": "spec-body p-keyfeatures"})  
    ul = indiv_prod_spec.find("ul", {"class": "dtls-list clear"})
    li = ul.find_all("li", {"class": "col-xs-8 dtls-li"})
    for i in li:
        print(str(i.text).split("\n"), str(i.text).split("\n"))
        if "RAM:" in i.text: 
            try: details['ram'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['ram'] = ""
        elif "Screen Size (in cm):" in i.text: 
            try: details['screen'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['screen'] = ""
        elif "Rear Camera:" in i.text: 
            try: details['rear_cam'] = str(i.text).split("\n")[-2].split(":")[1]        
            except: details['rear_cam'] = ""
        elif "Front Camera:" in i.text: 
            try: details['front_cam'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['front_cam'] = ""
        elif "Internal Memory:" in i.text:
            try: details['memory'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['memory'] = ""
        elif "Battery Capacity (in mAh):" in i.text: 
            try: details['battery'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['battery'] = ""
        elif "Processor Cores:" in i.text: 
            try: details['processor'] = str(i.text).split("\n")[-2].split(":")[1]
            except: details['processor'] = ""
        else: pass
    print(details)
    writer.writerow({'Prod Name': details['prod_name'], 'Color': details['color'], 'Rating': details['rating'], 'Price': details['price'],
                     'RAM': details['ram'], 'Screen': details['screen'], 'Rear Camera': details['rear_cam'], 
                     'Front Camera': details['front_cam'], 'Memory': details['memory'], 'Battery': details['battery'], 
                     'Processor': details['processor']})
driver.quit()
file_object.close()
print("CSV creation done!!")

https://www.snapdeal.com/product/redmi-9-prime-64gb-4/6917529666173669200#bcrumbLabelId:175
['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):16.51 cm (6.5)', ''] ['', '', 'Screen Size (in cm):16.51 cm (6.5)', '']
['', '', 'Rear Camera:13MP+8MP+2MP', ''] ['', '', 'Rear Camera:13MP+8MP+2MP', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Internal Memory:64GB', ''] ['', '', 'Internal Memory:64GB', '']
['', '', 'Battery Capacity (in mAh):5020', ''] ['', '', 'Battery Capacity (in mAh):5020', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL114088275', ''] ['', '', 'SUPC: SDL114088275', '']
{'prod_name': 'Redmi 9 Prime ( 64GB , 4 GB )', 'color': 'Blue', 'rating': '4.5', 'price': '10799', 'ram': '4 GB', 'screen': '16.51 cm (6.5)', 'rear_cam': '13MP+8MP+2MP', 'front_cam': '8 MP', 'memory': '64GB', 'battery': '5020', 'processor': 'Octa'}
https://www.snapdeal.com/product/itel-a25-pro

['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):15.74 cm (6.2)', ''] ['', '', 'Screen Size (in cm):15.74 cm (6.2)', '']
['', '', 'Rear Camera:13 MP', ''] ['', '', 'Rear Camera:13 MP', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Internal Memory:64GB', ''] ['', '', 'Internal Memory:64GB', '']
['', '', 'Battery Capacity (in mAh):4230', ''] ['', '', 'Battery Capacity (in mAh):4230', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL717313294', ''] ['', '', 'SUPC: SDL717313294', '']
{'prod_name': 'OPPO MOBILE OPPO A5S ( 64GB , 4 GB )', 'color': 'Green', 'rating': 'Currently Unavailable', 'price': '11990', 'ram': '4 GB', 'screen': '15.74 cm (6.2)', 'rear_cam': '13 MP', 'front_cam': '8 MP', 'memory': '64GB', 'battery': '4230', 'processor': 'Octa'}
https://www.snapdeal.com/product/redmi-8-64gb-4-gb/8070451181977327869#bcrumbLabelId:175
['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB

['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):13.97 cm (5.5)', ''] ['', '', 'Screen Size (in cm):13.97 cm (5.5)', '']
['', '', 'Screen Size (in cm):13.97 cm (5.5)', ''] ['', '', 'Screen Size (in cm):13.97 cm (5.5)', '']
['', '', 'Rear Camera:21 MP', ''] ['', '', 'Rear Camera:21 MP', '']
['', '', 'Rear Camera:21 MP', ''] ['', '', 'Rear Camera:21 MP', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Internal Memory:32GB', ''] ['', '', 'Internal Memory:32GB', '']
['', '', 'Internal Memory:32GB', ''] ['', '', 'Internal Memory:32GB', '']
['', '', 'Battery Capacity (in mAh):3000', ''] ['', '', 'Battery Capacity (in mAh):3000', '']
['', '', 'Battery Capacity (in mAh):3000', ''] ['', '', 'Battery Capacity (in mAh):3000', '']
['', '', 'Processor Cores:Quad', ''] ['', '', 'Processor Cores:Quad', '']
['', '', 'Proc

['', '', 'RAM:256 MB', ''] ['', '', 'RAM:256 MB', '']
['', '', 'Screen Size (in cm):4.826 cm (1.9)', ''] ['', '', 'Screen Size (in cm):4.826 cm (1.9)', '']
['', '', 'Rear Camera:2 MP', ''] ['', '', 'Rear Camera:2 MP', '']
['', '', 'Front Camera:No', ''] ['', '', 'Front Camera:No', '']
['', '', 'Internal Memory:256 MB', ''] ['', '', 'Internal Memory:256 MB', '']
['', '', 'Battery Capacity (in mAh):1000', ''] ['', '', 'Battery Capacity (in mAh):1000', '']
['', '', 'Processor Cores:Dual', ''] ['', '', 'Processor Cores:Dual', '']
['', '', 'SUPC: SDL219710909', ''] ['', '', 'SUPC: SDL219710909', '']
{'prod_name': 'Karbonn k310n ( 256 MB , 256 MB )', 'color': 'Blue', 'rating': '3.2', 'price': '849', 'ram': '256 MB', 'screen': '4.826 cm (1.9)', 'rear_cam': '2 MP', 'front_cam': 'No', 'memory': '256 MB', 'battery': '1000', 'processor': 'Dual'}
https://www.snapdeal.com/product/redmi-redmi-9-prime-128gb/5764608150757948183
['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (i

['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):16.51 cm (6.5)', ''] ['', '', 'Screen Size (in cm):16.51 cm (6.5)', '']
['', '', 'Rear Camera:48MP+8MP+2MP+2MP', ''] ['', '', 'Rear Camera:48MP+8MP+2MP+2MP', '']
['', '', 'Front Camera:13 MP', ''] ['', '', 'Front Camera:13 MP', '']
['', '', 'Internal Memory:64GB', ''] ['', '', 'Internal Memory:64GB', '']
['', '', 'Battery Capacity (in mAh):5020 mAh', ''] ['', '', 'Battery Capacity (in mAh):5020 mAh', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL779045770', ''] ['', '', 'SUPC: SDL779045770', '']
{'prod_name': 'Redmi Note 9 ( 64GB , 4 GB )', 'color': 'White', 'rating': '4.4', 'price': '13990', 'ram': '4 GB', 'screen': '16.51 cm (6.5)', 'rear_cam': '48MP+8MP+2MP+2MP', 'front_cam': '13 MP', 'memory': '64GB', 'battery': '5020 mAh', 'processor': 'Octa'}
https://www.snapdeal.com/product/redmi-8-64gb-4-gb/6917529649149840219
['', '', 'RAM:4 GB', ''] ['', '', 'RAM:

...

['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):16.25 cm (6.4)', ''] ['', '', 'Screen Size (in cm):16.25 cm (6.4)', '']
['', '', 'Rear Camera:48M+8M+2M+2M', ''] ['', '', 'Rear Camera:48M+8M+2M+2M', '']
['', '', 'Front Camera:16 MP', ''] ['', '', 'Front Camera:16 MP', '']
['', '', 'Internal Memory:128GB', ''] ['', '', 'Internal Memory:128GB', '']
['', '', 'Battery Capacity (in mAh):4000mAh', ''] ['', '', 'Battery Capacity (in mAh):4000mAh', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL284440138', ''] ['', '', 'SUPC: SDL284440138', '']
{'prod_name': 'Oppo F15 ( 128GB , 4 GB )', 'color': 'Blue', 'rating': '3.7', 'price': '16990', 'ram': '4 GB', 'screen': '16.25 cm (6.4)', 'rear_cam': '48M+8M+2M+2M', 'front_cam': '16 MP', 'memory': '128GB', 'battery': '4000mAh', 'processor': 'Octa'}
https://www.snapdeal.com/product/itel-vision-1-32gb-2/8070451217409983808
['', '', 'RAM:2 GB', ''] ['', '', 'RAM:2 GB', '']
['

['', '', 'RAM:4 GB', ''] ['', '', 'RAM:4 GB', '']
['', '', 'Screen Size (in cm):15.49 cm (6.1)', ''] ['', '', 'Screen Size (in cm):15.49 cm (6.1)', '']
['', '', 'Rear Camera:2 MP', ''] ['', '', 'Rear Camera:2 MP', '']
['', '', 'Front Camera:13 MP', ''] ['', '', 'Front Camera:13 MP', '']
['', '', 'Internal Memory:64GB', ''] ['', '', 'Internal Memory:64GB', '']
['', '', 'Battery Capacity (in mAh):4000 mAh', ''] ['', '', 'Battery Capacity (in mAh):4000 mAh', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL509615319', ''] ['', '', 'SUPC: SDL509615319', '']
{'prod_name': 'Lenovo Lenovo L19111 ( 64GB , 4 GB )', 'color': 'Blue', 'rating': 'Currently Unavailable', 'price': '8999', 'ram': '4 GB', 'screen': '15.49 cm (6.1)', 'rear_cam': '2 MP', 'front_cam': '13 MP', 'memory': '64GB', 'battery': '4000 mAh', 'processor': 'Octa'}
https://www.snapdeal.com/product/oppo-cph2015-128gb-6-gb/6917529713199791079
['', '', 'RAM:6 GB', ''] ['', '', 'RAM:6 GB',

['', '', 'RAM:6 GB', ''] ['', '', 'RAM:6 GB', '']
['', '', 'Screen Size (in cm):16.51 cm (6.5)', ''] ['', '', 'Screen Size (in cm):16.51 cm (6.5)', '']
['', '', 'Rear Camera:12M+2M+2M', ''] ['', '', 'Rear Camera:12M+2M+2M', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Internal Memory:128GB', ''] ['', '', 'Internal Memory:128GB', '']
['', '', 'Battery Capacity (in mAh):4230mAh', ''] ['', '', 'Battery Capacity (in mAh):4230mAh', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL237036210', ''] ['', '', 'SUPC: SDL237036210', '']
{'prod_name': 'OPPO A31 (128 GB , 6GB )', 'color': 'Green', 'rating': '3.6', 'price': '14990', 'ram': '6 GB', 'screen': '16.51 cm (6.5)', 'rear_cam': '12M+2M+2M', 'front_cam': '8 MP', 'memory': '128GB', 'battery': '4230mAh', 'processor': 'Octa'}
https://www.snapdeal.com/product/nokia-nokia-3-16gb/8646911943061116645
['', '', 'RAM:2 GB', ''] ['', '', 'RAM:2 GB', '']
['', '', 'Screen

['', '', 'RAM:2 GB', ''] ['', '', 'RAM:2 GB', '']
['', '', 'Screen Size (in cm):15.74 cm (6.2)', ''] ['', '', 'Screen Size (in cm):15.74 cm (6.2)', '']
['', '', 'Rear Camera:13MP+2MP', ''] ['', '', 'Rear Camera:13MP+2MP', '']
['', '', 'Front Camera:8 MP', ''] ['', '', 'Front Camera:8 MP', '']
['', '', 'Internal Memory:32GB', ''] ['', '', 'Internal Memory:32GB', '']
['', '', 'Battery Capacity (in mAh):5000', ''] ['', '', 'Battery Capacity (in mAh):5000', '']
['', '', 'Processor Cores:Octa', ''] ['', '', 'Processor Cores:Octa', '']
['', '', 'SUPC: SDL006602093', ''] ['', '', 'SUPC: SDL006602093', '']
{'prod_name': 'Redmi 8A Dual ( 32GB , 2 GB )', 'color': 'White', 'rating': '4.7', 'price': '7599', 'ram': '2 GB', 'screen': '15.74 cm (6.2)', 'rear_cam': '13MP+2MP', 'front_cam': '8 MP', 'memory': '32GB', 'battery': '5000', 'processor': 'Octa'}
https://www.snapdeal.com/product/oppo-f17-128gb-8-gb/5764608150291902554
['', '', 'RAM:8 GB', ''] ['', '', 'RAM:8 GB', '']
['', '', 'Screen Size (in 

['', '', 'RAM:8 GB', ''] ['', '', 'RAM:8 GB', '']
['', '', 'Screen Size (in cm):16.25 cm (6.4)', ''] ['', '', 'Screen Size (in cm):16.25 cm (6.4)', '']
['', '', 'Rear Camera:16MP+8MP+2MP', ''] ['', '', 'Rear Camera:16MP+8MP+2MP', '']
['', '', 'Front Camera:16 MP', ''] ['', '', 'Front Camera:16 MP', '']
['', '', 'Internal Memory:128GB', ''] ['', '', 'Internal Memory:128GB', '']
['', '', 'Battery Capacity (in mAh):4015 mAh', ''] ['', '', 'Battery Capacity (in mAh):4015 mAh', '']
['', '', 'Processor Cores:Quad', ''] ['', '', 'Processor Cores:Quad', '']
['', '', 'SUPC: SDL455366144', ''] ['', '', 'SUPC: SDL455366144', '']
{'prod_name': 'Oppo F17 ( 128GB , 8 GB )', 'color': 'Orange', 'rating': 'Currently Unavailable', 'price': '18490', 'ram': '8 GB', 'screen': '16.25 cm (6.4)', 'rear_cam': '16MP+8MP+2MP', 'front_cam': '16 MP', 'memory': '128GB', 'battery': '4015 mAh', 'processor': 'Quad'}
https://www.snapdeal.com/product/oppo-a53-64gb-4-gb/5764608189019936786
['', '', 'RAM:4 GB', ''] ['', '

In [6]:
file_object.close()